# NYC Food Rescue Stochastic Optimization - Visualizations

Visualizations for the two-stage stochastic model:
- **First stage**: Which distribution centers to open
- **Second stage**: Flow decisions under demand scenarios (2023, 2024, 2025)

In [ ]:
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import folium
from folium.plugins import MarkerCluster
from IPython.display import display, HTML

## Load Data

In [ ]:
pareto_df = pd.read_csv('viz_data/pareto_results.csv')
restaurants_df = pd.read_csv('viz_data/restaurants.csv')
centers_df = pd.read_csv('viz_data/donation_centers.csv')
neighborhoods_df = pd.read_csv('viz_data/neighborhoods.csv')
scenario_summary = pd.read_csv('viz_data/scenario_summary.csv')
deterministic_benchmark = pd.read_csv("viz_data/baseline_deterministic_solution.csv")

try:
    centers_opened = pd.read_csv('viz_data/centers_opened.csv')
    flows_df = pd.read_csv('viz_data/flows_2023.csv')
    alloc_2023 = pd.read_csv('viz_data/allocations_2023.csv')
    alloc_2024 = pd.read_csv('viz_data/allocations_2024.csv')
    alloc_2025 = pd.read_csv('viz_data/allocations_2025.csv')
except:
    centers_opened, flows_df = None, None
    alloc_2023, alloc_2024, alloc_2025 = None, None, None

print(f"Pareto solutions: {len(pareto_df)}")
print(f"Restaurants: {len(restaurants_df)}")
print(f"Centers: {len(centers_df)}")
print(f"Neighborhoods: {len(neighborhoods_df)}")

## Color Theme

In [ ]:
COLORS = {
    'background': '#0a0a0f',
    'surface': '#14141f',
    'primary': '#ff6b35',
    'secondary': '#4ecdc4',
    'accent': '#ffe66d',
    'purple': '#a855f7',
    'text': '#f7f7f7',
    'text_muted': '#8b8b9e',
    'grid': '#2a2a3e',
    'scenario_2023': '#ff6b35',
    'scenario_2024': '#4ecdc4',
    'scenario_2025': '#a855f7'
}

# General Map of the distribution centers, restaurants and neighborhood in NYC

In [ ]:
# Create map
m = folium.Map(location=[40.7128, -73.9560], zoom_start=11, tiles='CartoDB positron')

# Restaurants (orange circles, clustered)
restaurant_cluster = MarkerCluster(name='Restaurants (Supply)')
for _, row in restaurants_df.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5,
        color='#ff6b35',
        fill=True,
        fill_color='#ff6b35',
        fill_opacity=0.8,
        popup=f"Restaurant {int(row['id'])}<br>Supply: {row['supply']:,.0f} lbs"
    ).add_to(restaurant_cluster)
restaurant_cluster.add_to(m)

# Distribution Centers (green squares)
for _, row in centers_df.iterrows():
    folium.RegularPolygonMarker(
        location=[row['latitude'], row['longitude']],
        number_of_sides=4,
        radius=8,
        color='#2ecc71',
        fill=True,
        fill_color='#2ecc71',
        fill_opacity=0.9,
        popup=f"Distribution Center {int(row['id'])}"
    ).add_to(m)

# Neighborhoods (purple circles, sized by demand)
demand_col = 'demand_2023' if 'demand_2023' in neighborhoods_df.columns else 'demand'
demand_nbhd = neighborhoods_df[neighborhoods_df[demand_col] > 0]
max_demand = demand_nbhd[demand_col].max()

for _, row in demand_nbhd.iterrows():
    radius = 4 + 12 * (row[demand_col] / max_demand)
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=radius,
        color='#9b59b6',
        fill=True,
        fill_color='#9b59b6',
        fill_opacity=0.5,
        popup=f"Neighborhood {int(row['id'])}<br>Demand: {row[demand_col]:,.0f} lbs"
    ).add_to(m)

# Add legend
legend_html = '''
<div style="position: fixed; 
            bottom: 30px; left: 30px; 
            background-color: white;
            border: 2px solid #333;
            border-radius: 8px;
            padding: 12px 16px;
            z-index: 9999;
            font-family: Arial, sans-serif;
            font-size: 13px;
            box-shadow: 2px 2px 6px rgba(0,0,0,0.3);">
    <b style="font-size: 14px;">Legend</b><br><br>
    <span style="color: #ff6b35;">●</span> Restaurants (Supply)<br>
    <span style="color: #2ecc71;">■</span> Distribution Centers<br>
    <span style="color: #9b59b6;">●</span> Neighborhoods (Demand)
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Add layer control
folium.LayerControl().add_to(m)

# Save
m.save('figure1_network_locations.html')
print('Saved: figure1_network_locations.html')
m

---
## 1. Pareto Frontier (Cost vs. Equity)

In [ ]:
df = pareto_df.sort_values('avg_transport_cost').copy()

fig = go.Figure()

# Pareto line
fig.add_trace(go.Scatter(
    x=df['avg_transport_cost'], y=df['avg_equity_t'],
    mode='lines',
    line=dict(color=COLORS['primary'], width=3, shape='spline', smoothing=0.8),
    hoverinfo='skip',
    name='Pareto Frontier'
))

# Points with hover info
hover_text = [f"<b>w_cost={row.w_cost:.1f}, w_eq={row.w_eq:.1f}</b><br>" +
              f"Avg Transport Cost: {row.avg_transport_cost:,.0f}<br>" +
              f"Avg Worst Unmet: {row.avg_equity_t:,.0f}<br>" +
              f"Centers Opened: {row.num_centers}<br>" +
              f"Avg Delivered: {row.avg_total_recv:,.0f}"
              for _, row in df.iterrows()]

fig.add_trace(go.Scatter(
    x=df['avg_transport_cost'], y=df['avg_equity_t'],
    mode='markers',
    marker=dict(
        size=10,
        color=df['num_centers'],
        colorscale='Viridis',
        colorbar=dict(title='Centers<br>Opened'),
        line=dict(color='white', width=1)
    ),
    text=hover_text,
    hovertemplate='%{text}<extra></extra>',
    name='Solutions'
))

# Annotations for extremes
cost_opt = df.loc[df['avg_transport_cost'].idxmin()]
eq_opt = df.loc[df['avg_equity_t'].idxmin()]

fig.add_annotation(x=cost_opt['avg_transport_cost'], y=cost_opt['avg_equity_t'],
                   text="Cost Optimal", showarrow=True, arrowcolor=COLORS['secondary'],
                   font=dict(color=COLORS['secondary'], size=11), ax=-60, ay=-30)
fig.add_annotation(x=eq_opt['avg_transport_cost'], y=eq_opt['avg_equity_t'],
                   text="Equity Optimal", showarrow=True, arrowcolor=COLORS['accent'],
                   font=dict(color=COLORS['accent'], size=11), ax=60, ay=30)

fig.update_layout(
    title=dict(text="<b>Pareto Frontier: Cost vs. Equity</b><br><sub>Stochastic Model (3 Demand Scenarios)</sub>", 
               x=0.5, font=dict(size=18)),
    xaxis=dict(title="Avg. Transportation Cost", tickformat=',', gridcolor='#e0e0e0'),
    yaxis=dict(title="Avg. Worst Unmet Demand", tickformat=',', gridcolor='#e0e0e0'),
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(color='#333333'),
    showlegend=False,
    height=550
)

fig.show()

In [ ]:
# output the weights
# weights will be used for the first two models

---
## 2. Scenario Comparison: Worst Unmet Demand by Year

In [ ]:
# Select a few representative solutions
df_sorted = pareto_df.sort_values('avg_equity_t')
selected_idx = [0, len(df_sorted)//4, len(df_sorted)//2, 3*len(df_sorted)//4, len(df_sorted)-1]
selected = df_sorted.iloc[selected_idx].reset_index(drop=True)

fig = go.Figure()

x_labels = [f"w_c={row.w_cost:.1f}\nw_e={row.w_eq:.1f}" for _, row in selected.iterrows()]

fig.add_trace(go.Bar(name='2023', x=x_labels, y=selected['t_2023'], marker_color=COLORS['scenario_2023']))
fig.add_trace(go.Bar(name='2024', x=x_labels, y=selected['t_2024'], marker_color=COLORS['scenario_2024']))
fig.add_trace(go.Bar(name='2025', x=x_labels, y=selected['t_2025'], marker_color=COLORS['scenario_2025']))

fig.update_layout(
    title=dict(text="<b>Worst Unmet Demand by Scenario</b>", x=0.5, font=dict(size=16, color=COLORS['text'])),
    xaxis=dict(title="Solution Weights", color=COLORS['text_muted']),
    yaxis=dict(title="Worst Unmet Demand (lbs)", tickformat=',', gridcolor=COLORS['grid'], color=COLORS['text_muted']),
    barmode='group',
    plot_bgcolor=COLORS['surface'],
    paper_bgcolor=COLORS['background'],
    font=dict(color=COLORS['text']),
    legend=dict(title='Scenario', orientation='h', y=1.1, x=0.5, xanchor='center'),
    height=450
)

fig.show()

## Map
## Distribution of unmet deamnd (Visual of changing unmet demand overtime, number of miles driven, distance)

---
## 3. Centers Opened vs. Objective Trade-off

In [ ]:
fig = make_subplots(rows=1, cols=2, subplot_titles=('Centers vs. Transport Cost', 'Centers vs. Equity'))

fig.add_trace(go.Scatter(
    x=pareto_df['num_centers'], y=pareto_df['avg_transport_cost'],
    mode='markers',
    marker=dict(size=10, color=COLORS['primary'], opacity=0.7),
    hovertemplate='Centers: %{x}<br>Cost: %{y:,.0f}<extra></extra>'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=pareto_df['num_centers'], y=pareto_df['avg_equity_t'],
    mode='markers',
    marker=dict(size=10, color=COLORS['secondary'], opacity=0.7),
    hovertemplate='Centers: %{x}<br>Worst Unmet: %{y:,.0f}<extra></extra>'
), row=1, col=2)

fig.update_layout(
    title=dict(text="<b>Impact of Number of Centers Opened</b>", x=0.5, font=dict(size=16, color=COLORS['text'])),
    plot_bgcolor=COLORS['surface'],
    paper_bgcolor=COLORS['background'],
    font=dict(color=COLORS['text']),
    showlegend=False,
    height=400
)
fig.update_xaxes(title_text='Centers Opened', gridcolor=COLORS['grid'])
fig.update_yaxes(gridcolor=COLORS['grid'], tickformat=',')

fig.show()

---
## 4. Scenario Summary (Bar Chart)

In [ ]:
fig = make_subplots(rows=1, cols=2, subplot_titles=('Total Demand',  'Coverage %'))

colors = [COLORS['scenario_2023'], COLORS['scenario_2024'], COLORS['scenario_2025']]

fig.add_trace(go.Bar(
    x=scenario_summary['scenario'], y=scenario_summary['total_demand'],
    marker_color=colors, showlegend=False
), row=1, col=1)

coverage = 100 * scenario_summary['total_received'] / scenario_summary['total_demand']
fig.add_trace(go.Bar(
    x=scenario_summary['scenario'], y=coverage,
    marker_color=colors, showlegend=False,
    text=[f"{v:.1f}%" for v in coverage], textposition='outside'
), row=1, col=2)

fig.update_layout(
    title=dict(text="<b>Scenario Performance Summary</b>", x=0.5, font=dict(size=16, color=COLORS['text'])),
    plot_bgcolor=COLORS['surface'],
    paper_bgcolor=COLORS['background'],
    font=dict(color=COLORS['text']),
    height=400
)
fig.update_yaxes(gridcolor=COLORS['grid'], tickformat=',')

fig.show()

In [ ]:
# First-Stage Analysis: Distribution Center Opening Decisions

# Ensure centers_opened is merged with centers_df
centers_merged = centers_df.merge(centers_opened, on='id')

# Assign boroughs based on latitude/longitude boundaries (approximate)
def assign_borough(lat, lon):
    if lat > 40.8:
        return 'Bronx'
    elif lon < -73.95 and lat > 40.7:
        return 'Manhattan'
    elif lon > -73.95 and lat > 40.65:
        return 'Queens'
    elif lat < 40.65:
        return 'Staten Island' if lon < -74.05 else 'Brooklyn'
    else:
        return 'Brooklyn'

centers_merged['borough'] = centers_merged.apply(lambda r: assign_borough(r['latitude'], r['longitude']), axis=1)

# =============================================================================
# FIGURE A: Centers Opened by Borough (Grouped Bar Chart)
# =============================================================================

borough_summary = centers_merged.groupby('borough').agg(
    total=('id', 'count'),
    opened=('opened', 'sum')
).reset_index()
borough_summary['closed'] = borough_summary['total'] - borough_summary['opened']

fig_borough = go.Figure()

fig_borough.add_trace(go.Bar(
    name='Opened',
    x=borough_summary['borough'],
    y=borough_summary['opened'],
    marker_color='#2ecc71',
    text=borough_summary['opened'].astype(int),
    textposition='outside'
))

fig_borough.add_trace(go.Bar(
    name='Not Opened',
    x=borough_summary['borough'],
    y=borough_summary['closed'],
    marker_color='#95a5a6',
    text=borough_summary['closed'].astype(int),
    textposition='outside'
))

fig_borough.update_layout(
    title=dict(text='<b>First-Stage Decision: Distribution Centers by Borough</b>', x=0.5),
    xaxis_title='Borough',
    yaxis_title='Number of Centers',
    barmode='group',
    template='plotly_white',
    height=500,
    width=800
)

# fig_borough.write_image('images/centers_by_borough.png', scale=2)
fig_borough.show()


# =============================================================================
# FIGURE B: Geographic Scatter - Opened vs Closed Centers
# =============================================================================

fig_geo = go.Figure()

# Closed centers (gray X)
closed = centers_merged[centers_merged['opened'] == 0]
opened = centers_merged[centers_merged['opened'] == 1]

fig_geo.add_trace(go.Scatter(
    x=closed['longitude'],
    y=closed['latitude'],
    mode='markers',
    name=f'Not Opened (n={len(closed)})',
    marker=dict(color='#bdc3c7', size=10, symbol='x')
))

# Opened centers (green circle)
fig_geo.add_trace(go.Scatter(
    x=opened['longitude'],
    y=opened['latitude'],
    mode='markers',
    name=f'Opened (n={len(opened)})',
    marker=dict(color='#27ae60', size=12, symbol='circle', line=dict(color='white', width=1))
))

fig_geo.update_layout(
    title=dict(text='<b>First-Stage Decision: Geographic Distribution of Opened Centers</b>', x=0.5),
    xaxis_title='Longitude',
    yaxis_title='Latitude',
    template='plotly_white',
    height=700,
    width=800,
    yaxis=dict(scaleanchor='x', scaleratio=1),
    legend=dict(yanchor='top', y=0.99, xanchor='left', x=0.01)
)

# fig_geo.write_image('images/centers_geographic.png', scale=2)
fig_geo.show()


# =============================================================================
# SUMMARY STATISTICS
# =============================================================================
print("\n" + "="*50)
print("FIRST-STAGE DECISION SUMMARY")
print("="*50)
print(f"Total potential centers: {len(centers_merged)}")
print(f"Centers opened: {int(centers_merged['opened'].sum())} ({100*centers_merged['opened'].mean():.1f}%)")
print(f"\nBy Borough:")
print(borough_summary.to_string(index=False))

---
## 5. Network Map with Opened Centers

In [ ]:
m = folium.Map(location=[40.7128, -73.9560], zoom_start=11, tiles='CartoDB dark_matter')

# Restaurants (clustered)
restaurant_cluster = MarkerCluster(name='🍽️ Restaurants')
for _, row in restaurants_df.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=4, color='#ff6b35', fill=True, fill_opacity=0.7,
        popup=f"Restaurant {int(row['id'])}<br>Supply: {row['supply']:,.0f}"
    ).add_to(restaurant_cluster)
restaurant_cluster.add_to(m)

# Centers (opened vs closed)
if centers_opened is not None:
    centers_merged = centers_df.merge(centers_opened, on='id')
    for _, row in centers_merged.iterrows():
        color = 'green' if row['opened'] == 1 else 'gray'
        icon = 'check' if row['opened'] == 1 else 'times'
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            icon=folium.Icon(color=color, icon=icon, prefix='fa'),
            popup=f"Center {int(row['id'])}<br>{'OPEN' if row['opened']==1 else 'Closed'}"
        ).add_to(m)
else:
    for _, row in centers_df.iterrows():
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            icon=folium.Icon(color='green', icon='archive', prefix='fa'),
            popup=f"Center {int(row['id'])}"
        ).add_to(m)

# Neighborhoods with demand (using 2023)
demand_col = 'demand_2023' if 'demand_2023' in neighborhoods_df.columns else 'demand'
demand_nbhd = neighborhoods_df[neighborhoods_df[demand_col] > 0]
max_demand = demand_nbhd[demand_col].max()

for _, row in demand_nbhd.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3 + 12 * (row[demand_col] / max_demand),
        color='#a855f7', fill=True, fill_opacity=0.5,
        popup=f"Neighborhood {int(row['id'])}<br>Demand 2023: {row[demand_col]:,.0f}"
    ).add_to(m)

# Flows
if flows_df is not None and len(flows_df) > 0:
    rest_coords = dict(zip(restaurants_df['id'], zip(restaurants_df['latitude'], restaurants_df['longitude'])))
    center_coords = dict(zip(centers_df['id'], zip(centers_df['latitude'], centers_df['longitude'])))
    nbhd_coords = dict(zip(neighborhoods_df['id'], zip(neighborhoods_df['latitude'], neighborhoods_df['longitude'])))
    
    sig_flows = flows_df[flows_df['flow'] > 1000]
    max_flow = sig_flows['flow'].max() if len(sig_flows) > 0 else 1
    
    for _, row in sig_flows.iterrows():
        from_coord = rest_coords.get(row['from_id']) if row['from_type'] == 'restaurant' else center_coords.get(row['from_id'])
        to_coord = center_coords.get(row['to_id']) if row['to_type'] == 'center' else nbhd_coords.get(row['to_id'])
        if from_coord and to_coord:
            folium.PolyLine(
                [from_coord, to_coord],
                weight=1 + 3 * (row['flow'] / max_flow),
                color='#ffe66d', opacity=0.6
            ).add_to(m)

folium.LayerControl().add_to(m)
m.save('network_map.html')
print('Saved: network_map.html')
m

---
## 7. Allocation Distribution by Scenario

In [ ]:
# =============================================================================
# SCENARIO COMPARISON VISUALS
# =============================================================================

# --- FIGURE 1: Worst Unmet Demand by Scenario (for selected solutions) ---
df_sorted = pareto_df.sort_values('avg_equity_t')
selected_idx = [0, len(df_sorted)//4, len(df_sorted)//2, 3*len(df_sorted)//4, len(df_sorted)-1]
selected = df_sorted.iloc[selected_idx].reset_index(drop=True)

x_labels = [f"w_c={row.w_cost:.1f}, w_e={row.w_eq:.1f}" for _, row in selected.iterrows()]

fig1 = go.Figure()
fig1.add_trace(go.Bar(name='2023', x=x_labels, y=selected['t_2023'], marker_color='#ff6b35'))
fig1.add_trace(go.Bar(name='2024', x=x_labels, y=selected['t_2024'], marker_color='#4ecdc4'))
fig1.add_trace(go.Bar(name='2025', x=x_labels, y=selected['t_2025'], marker_color='#a855f7'))

fig1.update_layout(
    title=dict(text='<b>Worst Unmet Demand by Scenario</b>', x=0.5),
    xaxis_title='Solution Weights',
    yaxis_title='Worst Unmet Demand (lbs)',
    yaxis=dict(tickformat=','),
    barmode='group',
    template='plotly_white',
    legend=dict(title='Scenario', orientation='h', y=1.12, x=0.5, xanchor='center'),
    height=450, width=800
)
# fig1.write_image('images/scenario_worst_unmet.png', scale=2)
fig1.show()


# --- FIGURE 2: Total Demand, Received, Unmet by Scenario ---
fig2 = go.Figure()

scenarios = scenario_summary['scenario']
fig2.add_trace(go.Bar(name='Total Demand', x=scenarios, y=scenario_summary['total_demand'], marker_color='#3498db'))
fig2.add_trace(go.Bar(name='Total Received', x=scenarios, y=scenario_summary['total_received'], marker_color='#2ecc71'))
fig2.add_trace(go.Bar(name='Total Unmet', x=scenarios, y=scenario_summary['total_unmet'], marker_color='#e74c3c'))

fig2.update_layout(
    title=dict(text='<b>Demand Fulfillment by Scenario</b>', x=0.5),
    xaxis_title='Scenario (Year)',
    yaxis_title='Food (lbs)',
    yaxis=dict(tickformat=','),
    barmode='group',
    template='plotly_white',
    legend=dict(orientation='h', y=1.1, x=0.5, xanchor='center'),
    height=450, width=700
)
# fig2.write_image('images/scenario_fulfillment.png', scale=2)
fig2.show()


# --- FIGURE 3: Coverage Percentage by Scenario ---
scenario_summary['coverage_pct'] = 100 * scenario_summary['total_received'] / scenario_summary['total_demand']

fig3 = go.Figure()
fig3.add_trace(go.Bar(
    x=scenario_summary['scenario'],
    y=scenario_summary['coverage_pct'],
    marker_color=['#ff6b35', '#4ecdc4', '#a855f7'],
    text=[f"{v:.1f}%" for v in scenario_summary['coverage_pct']],
    textposition='outside'
))

fig3.update_layout(
    title=dict(text='<b>Demand Coverage by Scenario</b>', x=0.5),
    xaxis_title='Scenario (Year)',
    yaxis_title='Coverage (%)',
    yaxis=dict(range=[0, max(scenario_summary['coverage_pct']) * 1.15]),
    template='plotly_white',
    height=400, width=500
)
# fig3.write_image('images/scenario_coverage.png', scale=2)
fig3.show()

# --- Print summary stats ---
print("\nSCENARIO SUMMARY:")
print(scenario_summary.to_string(index=False))

In [ ]:
# Animated Bar Chart: Top Neighborhoods by Unmet Demand Across Pareto Frontier
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

# Load data
pareto_sorted = pareto_df.sort_values('avg_transport_cost').reset_index(drop=True)
neighborhoods_df = pd.read_csv('viz_data/neighborhoods.csv')

# =============================================================================
# ANIMATED BAR CHART: Top 20 neighborhoods by unmet demand
# =============================================================================

fig_animated = go.Figure()

frames = []
slider_steps = []
top_n = 20

# Get max unmet across all solutions for consistent x-axis
max_unmet_global = 0
for idx in range(len(pareto_sorted)):
    alloc = pd.read_csv(f'viz_data/pareto_allocations/alloc_{idx+1}.csv')
    max_unmet_global = max(max_unmet_global, alloc['avg_unmet'].max())

for idx, row in pareto_sorted.iterrows():
    alloc = pd.read_csv(f'viz_data/pareto_allocations/alloc_{idx+1}.csv')
    top_unmet = alloc.nlargest(top_n, 'avg_unmet').reset_index(drop=True)
    
    # Merge with neighborhood names
    top_merged = top_unmet.merge(neighborhoods_df[['id', 'name']], left_on='neighborhood_id', right_on='id')
    
    frame = go.Frame(
        data=[
            go.Bar(
                x=top_merged['avg_unmet'],
                y=top_merged['name'],
                orientation='h',
                marker=dict(
                    color=top_merged['avg_unmet'],
                    colorscale='Reds',
                    cmin=0,
                    cmax=max_unmet_global
                ),
                text=[f"{v:,.0f}" for v in top_merged['avg_unmet']],
                textposition='outside',
                textfont=dict(size=10)
            )
        ],
        name=str(idx),
        layout=go.Layout(
            title=dict(
                text=f"<b>Top {top_n} Neighborhoods by Unmet Demand</b><br>" +
                     f"<sub>Cost: {row.avg_transport_cost:,.0f} | " +
                     f"Worst Unmet: {row.avg_equity_t:,.0f}</sub>"
            )
        )
    )
    frames.append(frame)
    
    slider_steps.append(dict(
        args=[[str(idx)], dict(frame=dict(duration=500, redraw=True), mode='immediate')],
        label=f"{idx+1}",
        method='animate'
    ))

# Initial frame
initial_alloc = pd.read_csv('viz_data/pareto_allocations/alloc_1.csv')
initial_top = initial_alloc.nlargest(top_n, 'avg_unmet').reset_index(drop=True)
initial_merged = initial_top.merge(neighborhoods_df[['id', 'name']], left_on='neighborhood_id', right_on='id')
initial_row = pareto_sorted.iloc[0]

fig_animated.add_trace(go.Bar(
    x=initial_merged['avg_unmet'],
    y=initial_merged['name'],
    orientation='h',
    marker=dict(
        color=initial_merged['avg_unmet'],
        colorscale='Reds',
        cmin=0,
        cmax=max_unmet_global
    ),
    text=[f"{v:,.0f}" for v in initial_merged['avg_unmet']],
    textposition='outside',
    textfont=dict(size=10)
))

fig_animated.frames = frames

fig_animated.update_layout(
    title=dict(
        text=f"<b>Top {top_n} Neighborhoods by Unmet Demand</b><br>" +
             f"<sub>Cost: {initial_row.avg_transport_cost:,.0f} | " +
             f"Worst Unmet: {initial_row.avg_equity_t:,.0f}</sub>",
        x=0.5
    ),
    xaxis=dict(
        title='Unmet Demand (lbs)',
        tickformat=',',
        range=[0, max_unmet_global * 1.15]
    ),
    yaxis=dict(
        title='',
        autorange='reversed'
    ),
    template='plotly_white',
    height=600,
    width=900,
    margin=dict(l=250),  # More space for neighborhood names
    updatemenus=[
        dict(
            type='buttons',
            showactive=False,
            y=1.12,
            x=0.5,
            xanchor='center',
            buttons=[
                dict(label='▶ Play',
                     method='animate',
                     args=[None, dict(frame=dict(duration=600, redraw=True),
                                      fromcurrent=True,
                                      transition=dict(duration=300))]),
                dict(label='⏸ Pause',
                     method='animate',
                     args=[[None], dict(frame=dict(duration=0, redraw=False),
                                        mode='immediate',
                                        transition=dict(duration=0))])
            ]
        )
    ],
    sliders=[dict(
        active=0,
        yanchor='top',
        xanchor='left',
        currentvalue=dict(font=dict(size=12), prefix='Solution: ', visible=True, xanchor='center'),
        pad=dict(b=10, t=60),
        len=0.9,
        x=0.05,
        y=0,
        steps=slider_steps
    )]
)

fig_animated.write_html('images/top_unmet_neighborhoods_animated.html')
print('Saved: top_unmet_neighborhoods_animated.html')
fig_animated.show()


# =============================================================================
# STATIC: Side-by-side comparison (Cost-Optimal vs Equity-Optimal)
# =============================================================================

# Cost-optimal (first)
cost_alloc = pd.read_csv('viz_data/pareto_allocations/alloc_1.csv')
cost_top = cost_alloc.nlargest(top_n, 'avg_unmet').reset_index(drop=True)
cost_top_merged = cost_top.merge(neighborhoods_df[['id', 'name']], left_on='neighborhood_id', right_on='id')
cost_row = pareto_sorted.iloc[0]

# Equity-optimal (last)
eq_alloc = pd.read_csv(f'viz_data/pareto_allocations/alloc_{len(pareto_sorted)}.csv')
eq_top = eq_alloc.nlargest(top_n, 'avg_unmet').reset_index(drop=True)
eq_top_merged = eq_top.merge(neighborhoods_df[['id', 'name']], left_on='neighborhood_id', right_on='id')
eq_row = pareto_sorted.iloc[-1]

fig_static = make_subplots(
    rows=1, cols=2,
    subplot_titles=(
        f"Cost-Optimal<br>(Cost: {cost_row.avg_transport_cost:,.0f})",
        f"Equity-Optimal<br>(Cost: {eq_row.avg_transport_cost:,.0f})"
    ),
    horizontal_spacing=0.25
)

fig_static.add_trace(go.Bar(
    x=cost_top_merged['avg_unmet'],
    y=cost_top_merged['name'],
    orientation='h',
    marker_color='#e74c3c',
    text=[f"{v:,.0f}" for v in cost_top_merged['avg_unmet']],
    textposition='outside',
    textfont=dict(size=9),
    showlegend=False
), row=1, col=1)

fig_static.add_trace(go.Bar(
    x=eq_top_merged['avg_unmet'],
    y=eq_top_merged['name'],
    orientation='h',
    marker_color='#3498db',
    text=[f"{v:,.0f}" for v in eq_top_merged['avg_unmet']],
    textposition='outside',
    textfont=dict(size=9),
    showlegend=False
), row=1, col=2)

fig_static.update_layout(
    title=dict(text=f'<b>Top {top_n} Neighborhoods by Unmet Demand</b>', x=0.5),
    template='plotly_white',
    height=600,
    width=1200,
    margin=dict(l=200, r=50)
)

fig_static.update_xaxes(title_text='Unmet Demand (lbs)', tickformat=',', range=[0, max_unmet_global * 1.15])
fig_static.update_yaxes(autorange='reversed')

# fig_static.write_image('images/top_unmet_comparison.png', scale=2)
# print('Saved: top_unmet_comparison.png')
fig_static.show()

In [21]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import numpy as np

# # Load the scenario summary data
# scenario_summary = pd.read_csv("viz_data/scenario_summary.csv")

# # Example deterministic benchmark data (replace with your actual file)
# # deterministic_benchmark = pd.read_csv("viz_data/deterministic_benchmark.csv")
# # Or create manually from your table:
# deterministic_benchmark = pd.DataFrame({
#     'Year': [2023, 2024, 2025],
#     'ObjectiveValue': [7.07e6, 8.07e6, 8.18e6],
#     'TotalDistance': [6.69e6, 6.61e6, 7.19e6],
#     'WorstUnmetDemand': [191268.2, 729219.1, 495528.1]
# })

import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np



# ============================================================================
# Calculate Differences
# ============================================================================
comparison = pd.DataFrame({
    'Year': [2023, 2024, 2025],
    'Obj_Stochastic': scenario_summary['ObjectiveValue'],
    'Obj_Deterministic': deterministic_benchmark['ObjectiveValue'],
    'Obj_Diff_%': ((scenario_summary['ObjectiveValue'] - deterministic_benchmark['ObjectiveValue']) / 
                   deterministic_benchmark['ObjectiveValue'] * 100),
    'Dist_Stochastic': scenario_summary['TotalDistance'],
    'Dist_Deterministic': deterministic_benchmark['TotalDistance'],
    'Dist_Diff_%': ((scenario_summary['TotalDistance'] - deterministic_benchmark['TotalDistance']) / 
                    deterministic_benchmark['TotalDistance'] * 100),
    'Worst_Stochastic': scenario_summary['WorstUnmetDemand'],
    'Worst_Deterministic': deterministic_benchmark['WorstUnmetDemand'],
    'Worst_Diff_%': ((scenario_summary['WorstUnmetDemand'] - deterministic_benchmark['WorstUnmetDemand']) / 
                     deterministic_benchmark['WorstUnmetDemand'] * 100)
})

# Print comparison
print("="*80)
print("COMPARISON: Stochastic vs Deterministic")
print("="*80)
print(comparison.to_string(index=False))
print(f"\nAvg Objective Diff: {comparison['Obj_Diff_%'].mean():.2f}%")
print(f"Avg Distance Diff: {comparison['Dist_Diff_%'].mean():.2f}%")
print(f"Avg Worst Unmet Diff: {comparison['Worst_Diff_%'].mean():.2f}%\n")

# ============================================================================
# Plotly Visualization
# ============================================================================
years = ['2023', '2024', '2025']

fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=('Objective Value', 'Transportation Distance', 'Worst Unmet Demand'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}, {'type': 'bar'}]]
)

# Plot 1: Objective Value
fig.add_trace(go.Bar(name='Stochastic', x=years, y=scenario_summary['ObjectiveValue'], 
                     marker_color='#636EFA', showlegend=True), row=1, col=1)
fig.add_trace(go.Bar(name='Deterministic', x=years, y=deterministic_benchmark['ObjectiveValue'],
                     marker_color='#EF553B', showlegend=True), row=1, col=1)

# Plot 2: Total Distance
fig.add_trace(go.Bar(name='Stochastic', x=years, y=scenario_summary['TotalDistance'],
                     marker_color='#636EFA', showlegend=False), row=1, col=2)
fig.add_trace(go.Bar(name='Deterministic', x=years, y=deterministic_benchmark['TotalDistance'],
                     marker_color='#EF553B', showlegend=False), row=1, col=2)

# Plot 3: Worst Unmet Demand
fig.add_trace(go.Bar(name='Stochastic', x=years, y=scenario_summary['WorstUnmetDemand'],
                     marker_color='#636EFA', showlegend=False), row=1, col=3)
fig.add_trace(go.Bar(name='Deterministic', x=years, y=deterministic_benchmark['WorstUnmetDemand'],
                     marker_color='#EF553B', showlegend=False), row=1, col=3)

# Update layout
fig.update_layout(
    title_text="Stochastic vs Deterministic Model Comparison",
    height=500,
    showlegend=True,
    legend=dict(orientation="h", yanchor="bottom", y=1.05, xanchor="right", x=0.17)
)

fig.update_xaxes(title_text="Year", row=1, col=1)
fig.update_xaxes(title_text="Year", row=1, col=2)
fig.update_xaxes(title_text="Year", row=1, col=3)

fig.update_yaxes(title_text="Objective Value", row=1, col=1)
fig.update_yaxes(title_text="Distance", row=1, col=2)
fig.update_yaxes(title_text="Worst Unmet (lbs)", row=1, col=3)

fig.write_html('viz_data/stochastic_vs_deterministic_comparison.html')
fig.write_image('viz_data/stochastic_vs_deterministic_comparison.png', width=1400, height=500)

# Export comparison
comparison.to_csv('viz_data/model_comparison.csv', index=False)
print("Saved comparison to viz_data/model_comparison.csv and .html/.png")
fig.show()

COMPARISON: Stochastic vs Deterministic
 Year  Obj_Stochastic  Obj_Deterministic  Obj_Diff_%  Dist_Stochastic  Dist_Deterministic  Dist_Diff_%  Worst_Stochastic  Worst_Deterministic  Worst_Diff_%
 2023    6.696457e+06       7.068144e+06   -5.258617     4.967097e+06        6.685607e+06   -25.704626     189680.069500        191268.191411     -0.830312
 2024    7.866117e+06       8.071493e+06   -2.544458     5.043140e+06        6.613055e+06   -23.739631     736488.603807        729219.055925      0.996895
 2025    8.561409e+06       8.179989e+06    4.662846     5.914491e+06        7.188933e+06   -17.727832     648459.207663        495528.116828     30.862243

Avg Objective Diff: -1.05%
Avg Distance Diff: -22.39%
Avg Worst Unmet Diff: 10.34%

Saved comparison to viz_data/model_comparison.csv and .html/.png
